In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [4]:
from dotenv import load_dotenv
load_dotenv("../.env", override=False)

False

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [6]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
        "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [7]:
# Agent 2: Ligistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal="Manage all logistics for the event "
        "including catering and equipments",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [8]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
        "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [9]:
from pydantic import BaseModel
# Define a Pydantic model for venue details (demonstrating output as pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [10]:
venue_task = Task(
    description="Find a venue in {event_city} "
        "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
        "venue you found accomodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    agent=venue_coordinator
)

In [11]:
logistics_task = Task(
    description="Coordinate catering and "
        "equipment for an event "
        "with {expected_participants} participants "
        "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
        "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [12]:
marketing_task = Task(
    description="Promote the {event_topic} "
        "aiming to engage at least"
        "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
        "and attendee engagement formatted as markdown.",
    async_execution=False,
    output_file="marketing_report.md",
    agent=marketing_communications_agent
)

In [13]:
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],
    tasks=[venue_task,
           logistics_task,
           marketing_task],
    verbose=True
)

In [14]:
event_details = {
    "event_topic": "Tech Innovation Conference",
    "event_description": "A gathering of tech innovators "
        "and industry leaders "
        "to explore future technologies.",
    "event_city": "San Francisco",
    "tentative_date": "2024-09-15",
    "expected_participants": 500,
    "budget": 20000,
    "venue_type": "Conference Hall"
}

In [15]:
result=event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


# Agent: Venue Coordinator
## Thought: I should start by searching the internet for venues in San Francisco suitable for a Tech Innovation Conference. This will help me gather information on potential venues that meet the event's criteria.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"venues for events in San Francisco for tech innovation conference\"}"
## Tool Output: 
{'searchParameters': {'q': 'venues for events in San Francisco for tech innovation conference', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Top 12 tech conferences in San Francisco for 2024 - TravelPerk', 'link': 'https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/', 'snippet': "We've curated our list of the top 12 technology conferences, symposiums, and tech events taking place in San Francisco in 2024.", 'position': 1}, {'titl

 looks good


# Agent: Logistics Manager
## Task: Coordinate catering and equipment for an event with 500 participants on 2024-09-15.


# Agent: Logistics Manager
## Thought: I need to gather information about catering and equipment setup for the event with 500 participants on September 15, 2024. I will start by searching the internet for catering options and equipment rental services.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"catering services for events on September 15, 2024\"}"
## Tool Output: 
{'searchParameters': {'q': 'catering services for events on September 15, 2024', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Made by Meg Catering | Los Angeles Premier Catering Service', 'link': 'https://mbmcatering.com/', 'snippet': 'We cater a variety of events, including weddings (both offsite and at our property La Venta Inn), office & corporate events, birthday parties, Bar Mitzvahs, Bat ...', 'position': 1, 'sitelinks': [{'title': 'Wedding C

 I like it


# Agent: Marketing and Communications Agent
## Task: Promote the Tech Innovation Conference aiming to engage at least500 potential attendees.


# Agent: Marketing and Communications Agent
## Thought: I need to gather information on marketing activities and attendee engagement for the Tech Innovation Conference to create a comprehensive report.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"http://madebymegcatering.com\"}"
## Tool Output: 

Made by Meg Catering | Los Angeles Premier Catering Service
About
Press & Media
MBM Hospitality Blog
Weddings
Corporate
Bar Services
Management
La Venta
Venues
Contact
Los Angeles, CA
About
Press & Media
MBM Hospitality Blog
FAQ
Services
Weddings
Corporate
Bar Services
Management
LA Rec & Parks Bar Services
La Venta
LA Rec & Parks Bar Services
Venues
ESTD. 2007
Contact
Premier Los Angeles Event Catering
 â€“ Thoughtful Hospitality is Our Passion.
Elevated Catering for Weddings, Corporate Events, and More.
Discover Our Menus


In [16]:
from IPython.display import Markdown
Markdown(result.raw)

# Marketing Activities and Attendee Engagement Report for Tech Innovation Conference

## Marketing Activities:
1. **Conference Marketing Ideas**: Visit [this article](https://swoogo.events/blog/conference-marketing-ideas/) for 12 proven strategies to boost attendance and increase engagement.
2. **Event Technology Guide**: Find innovative tools to enhance engagement in [this comprehensive guide](https://taggbox.com/blog/event-technology/).
3. **Boosting Event Engagement Ideas**: Discover 29 proven ideas for increasing engagement with a virtual audience on [this website](https://spotme.com/blog/20-interactive-conference-ideas-to-increase-engagement-with-a-virtual-audience/).
4. **Event Success with Engagement Technology**: Learn about event gamification and other strategies to improve engagement using technology [here](https://machintel.com/blog/attendee-engagement-technology-for-successful-event/).

## Attendee Engagement:
1. **Strategies to Boost Attendee Engagement**: Check out [this article](https://eventtechlive.com/top-strategies-to-boost-attendee-engagement-at-your-next-event/) for top strategies and interactive technology ideas.
2. **Technology and Engagement**: Understand the role of event technology in enhancing engagement and experience on [this website](https://hub.theeventplannerexpo.com/attendee-engagement/the-role-of-event-technology-in-enhancing-attendee-engagement-and-experience).
3. **Enhancing Human Connection**: Explore insights on the future of event marketing focusing on human connection and data-driven strategies [here](https://samaaro.com/event-technology/the-future-of-event-marketing-a-focus-on-human-connection-and-data-driven-insights/).
4. **Attendee Engagement Tools**: Get over 20 ideas for boosting engagement at events on [Fliplet's blog](https://fliplet.com/blog/attendee-engagement/).

This report provides a glimpse into various marketing activities and engagement strategies that can be implemented for the upcoming Tech Innovation Conference to ensure a successful and engaging event.